In [6]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime
import json 

In [ ]:
file = "./NY/ny_p_neighbors11for_preprocessing.geojson"
state_boundary = gp.read_file(file)

In [ ]:
reader = csv.reader(open('./NY_county.csv', 'r'))
county_fips = {}
for row in reader:
    key = row[5]
    value = row[0]
    county_fips[key] = value

In [ ]:
state_boundary.head()

In [ ]:
#Rename columns
state_boundary.rename(columns = {"GEOID10":"id"}, inplace = True)
state_boundary.rename(columns = {"INTPTLAT10":"centerX"}, inplace = True)
state_boundary.rename(columns = {"INTPTLON10":"centerY"}, inplace = True)
state_boundary.rename(columns = {"NEIGHBORS":"neighbors"}, inplace = True)

#Add type columns
state_boundary.insert(1, 'canonical', None)
state_boundary.insert(2, 'type', 'ORIGINAL') 

#Drop column
#df = df.drop (columns = ['ALAND10'])
state_boundary.head()


In [ ]:
#Canonical name
STATE = 'NY'
for i, row in ny_boundary.iterrows():   
    id = state_boundary.at[i, 'id']
    county_id = id[ : 5]
    canonical_id = state + ' ' + county_fips[county_id] + ' ' + id[5:]
    state_boundary.at[i, 'canonical'] = canonical_id
    
    #print (canonicalId)
state_boundary.head()

In [ ]:
#Replace multipoly with polygon
arr_polygon = [0] * len(df)
arr_polygon = gp.GeoDataFrame (arr_polygon, columns = ['geometry'])
for index, row in df.iterrows():
    #Progress
    if index % 1000 == 0:
        print (index)
    polygon = row.geometry[0]
    arr_polygon.loc[index] = polygon
    
df['geometry'] = arr_polygon
df.head()    

In [ ]:
state_boundary.to_file("./NY/NY_precinct_boundary_neighbors_full.geojson", driver='GeoJSON')

In [35]:
### GENERATE NEIGHBORS

file = "./NY/ny_final.geojson"  
precincts = gp.read_file(file) # open file

precincts["neighbors"] = None  # add NEIGHBORS column

for index, row in precincts.iterrows():
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
        
    neighbors = precincts[~precincts.geometry.disjoint(row.geometry)].id.tolist()
    # remove own name from the list
    neighbors = [ name for name in neighbors if row.id != name ]
    # add names of neighbors as NEIGHBORS value
    precincts.at[index, 'neighbors'] = ", ".join(neighbors)
precincts.head()
#precincts.to_file(".NY/NY_neighbors.geojson", driver='GeoJSON')

14:50:57 0
14:51:38 1000
14:52:18 2000
14:52:59 3000
14:53:39 4000
14:54:19 5000
14:55:00 6000
14:55:40 7000
14:56:21 8000
14:57:01 9000
14:57:42 10000
14:58:23 11000
14:59:04 12000
14:59:46 13000
15:00:28 14000


,id,canonical,type,centerX,centerY,neighbors,geometry
0,36001263,NY Albany 263,ORIGINAL,+42.7272097,-073.7873353,"36001265, 36001262, 36001253, 36001247, 360012...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,NY Albany 232,ORIGINAL,+42.7295842,-073.7227157,"36001346, 36001251, 36001250, 36001344, 360013...","MULTIPOLYGON (((-73.71364 42.72799, -73.70998 ..."
2,36001342,NY Albany 342,ORIGINAL,+42.7348777,-073.7093285,"36001341, 36001346, 36001339, 36001340, 360012...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,NY Albany 341,ORIGINAL,+42.7355901,-073.7037615,"36001342, 36001339, 36001233, 36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,NY Albany 277,ORIGINAL,+42.7313271,-073.8385576,"36001276, 36001275, 36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [16]:
def create_neighbors_df(neighbors, df):
    neighbors = neighbors.split(',')
    neighbors_df = df[df['id'].isin(neighbors)][['id', 'geometry']]
    return neighbors_df 

In [14]:
#INTERSECTIONS

file = "./NY/NY.geojson"  
precincts = gp.read_file(file) # open file
precincts = precincts.drop(columns=['type', 'centerX', 'centerY', 'canonical'])
precincts.head()

,id,neighbors,geometry
0,36001263,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.71119 ..."
2,36001342,"36001233,36001243,36001340,36001341,36001339,3...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,"36001233,36001342,36001339,36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,"36001275,36001276,36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [17]:
AREA_THRESHOLD = 1e-4
intersections = gp.GeoDataFrame(crs = precincts.crs)

for index, row in precincts.iterrows():
    #For progress
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    neighbors = create_neighbors_df(row['neighbors'], precincts)
    #print(neighbors)
    # check if intersection occured
    overlaps = neighbors[neighbors.geometry.overlaps(row.geometry)]['id'].tolist()
    if len(overlaps) > 0:
        # compare the area with threshold
        for y in overlaps:
            temp_area = gp.overlay(precincts.loc[precincts.id == y,], precincts.loc[precincts.id == row.id], how='intersection')
            temp_area = temp_area.loc[temp_area.geometry.area >= AREA_THRESHOLD]
            
            if temp_area.shape[0] > 0:
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections], ignore_index=True),crs = precincts.crs)
                intersections['sorted'] = intersections.apply(lambda y: sorted([y['id_1'],y['id_2']]), axis=1)
                intersections['sorted'] = intersections.sorted.apply(lambda y: ''.join(y))
                intersections = intersections.drop_duplicates('sorted')
                intersections = intersections.reset_index()[['id_1','id_2','geometry']]
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections],ignore_index=True),crs=precincts.crs)

17:03:54 0
17:04:40 1000
17:05:41 2000
17:06:38 3000
17:07:26 4000
17:08:05 5000
17:08:59 6000
17:10:00 7000
17:10:40 8000
17:11:31 9000
17:12:35 10000
17:13:19 11000
17:14:14 12000
17:15:14 13000
17:16:18 14000


In [18]:
intersections.shape

(0, 0)

In [7]:
### GAPS

file = "./NY/ny_final.geojson"
precincts = gp.read_file(file)

precincts_diss = precincts[['geometry', 'type']]
precincts_diss = precincts_diss.dissolve(by='type')
#precincts_diss = df.drop (columns = ['type', 'id', 'canonical', 'centerX', 'centerY', 'area'])

precincts_diss.head()

,geometry
type,
ORIGINAL,"MULTIPOLYGON (((-74.25653 40.49296, -74.25909 ..."


In [8]:
interior = precincts_diss.interiors.values.tolist()

gaps = []
for i in interior:
    if i == []:
        continue
    #print(i[0].coords[:])    
    gaps.append(Polygon(i[0]))
    
gaps = gp.GeoDataFrame(geometry = gaps, crs = precincts.crs)
gaps['feature_touches'] = gaps.geometry.apply(lambda y: precincts.loc[precincts.touches(y)]['id'].tolist())
gaps.head()

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\array.py:575: UserWarning: Only Polygon objects have interior rings. For other geometry types, None is returned.
  "Only Polygon objects have interior rings. For other "


TypeError: 'NoneType' object is not subscriptable

In [16]:
## MAP COVERAGE ERROR

file_states = "./States/states_boundary.geojson"
state_boundary = gp.read_file(file_states)
state_boundary.head()

,id,centerX,centerY,area,geometry
0,PA,-77.6048035,41.1179355,12.7471968513065,"MULTIPOLYGON (((-80.51935 41.84956, -80.51922 ..."
1,KY,-85.76960750000001,37.82234,10.668081506716,"MULTIPOLYGON (((-88.06133 37.50533, -88.06368 ..."
2,NY,-75.77002999999999,42.7460485,15.58035125575651,"MULTIPOLYGON (((-79.31214 42.68680, -79.14347 ..."


In [20]:
file_precincts = "./NY/NY_final.geojson"
precinct_boundary = gp.read_file(file_precincts)
precinct_boundary.head(2)

,id,canonical,type,centerX,centerY,neighbors,geometry
0,36001263,NY Albany 263,ORIGINAL,+42.7272097,-073.7873353,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,NY Albany 232,ORIGINAL,+42.7295842,-073.7227157,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.70998 ..."


In [21]:
#Check difference between two areas
STATE = 'NY'
state = state_boundary[state_boundary['id'] == STATE]
state_area = state_boundary.at[1, 'area']
precinct_area = 0
for i, row in precinct_boundary.iterrows():
    precinct_area += row.geometry.area
#print(precinct_area)
print("Difference: ", float(state_area) - float(precinct_area))

Difference:  -4.911690818634073


In [22]:
precinct_dissolved = precinct_boundary[['geometry', 'type']]
precinct_dissolved = precinct_dissolved.dissolve(by='type')
precinct_dissolved.head()

,geometry
type,
ORIGINAL,"MULTIPOLYGON (((-74.25653 40.49296, -74.25909 ..."


In [23]:
difference = gp.overlay(state, precinct_dissolved, how = 'difference')

In [29]:
THRESHOLD = 5e-4

add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

ERROR_COVERAGE_TYPE = 'COVERAGE'
STATE = 'NY'
coverage_error_data = []

for multipolygon in difference.geometry:
    for polygon in multipolygon:
        #print(polygon.area)
        if polygon.area > THRESHOLD:
            bounds = polygon.bounds
            centerX = round((bounds[0] + bounds[2]) / 2, 6)
            centerY = round((bounds[1] + bounds[3]) / 2, 6)
            coverage_row = (ERROR_COVERAGE_TYPE, None, centerX, centerY, STATE)
            coverage_error_data.append(coverage_row)
            
#cursor.executemany(add_error_querry, coverage_error_data)
#mydb.commit()           
print(coverage_error_data)

[]


In [24]:
difference.to_file("./NY/NY_coverage_error.geojson", driver='GeoJSON')

In [31]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [30]:
file = "./NY/NY_final.geojson"
precincts = gp.read_file(file)
precincts.head()


,id,canonical,type,centerX,centerY,neighbors,geometry
0,36001263,NY Albany 263,ORIGINAL,+42.7272097,-073.7873353,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,NY Albany 232,ORIGINAL,+42.7295842,-073.7227157,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.70998 ..."
2,36001342,NY Albany 342,ORIGINAL,+42.7348777,-073.7093285,"36001233,36001243,36001340,36001341,36001339,3...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,NY Albany 341,ORIGINAL,+42.7355901,-073.7037615,"36001233,36001342,36001339,36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,NY Albany 277,ORIGINAL,+42.7313271,-073.8385576,"36001275,36001276,36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [42]:
add_precinct_querry = ("INSERT INTO mhaczynska.precinct "
               "(GEO_ID, NAME, STATE_NAME, CENTER_X, CENTER_Y, PRECINCT_TYPE, PRECINCT_GEOJSON) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s)")

update_precinct_querry = ("UPDATE mhaczynska.precinct SET PRECINCT_GEOJSON = %s WHERE GEO_ID = %s")

state = 'NY'
for index, row in precincts.iterrows():
    geometry_str = row.geometry.__geo_interface__
    geometry_str = json.dumps(geometry_str)     
    precinct_data = (row['id'], row['canonical'], state, row['centerX'], row['centerY'], row['type'], geometry_str)
    #update_precinct = (geometry_str, row['id'])
    #print(precinct_data)
    try:
        cursor.execute(add_precinct_querry, precinct_data)
        mydb.commit()
    except:
        pass
    

In [43]:
#ADD NEIGHBORS
add_neighbor_querry = ("INSERT INTO mhaczynska.neighbors "
               "(PRECINCT, NEIGHBOR_GEOID) "
               "VALUES (%s, %s)")

neighbor_data = []
for index, row in precincts.iterrows():
    neighbors = row['neighbors']
    neighbors = neighbors.split(",")    
    for neighbor in neighbors:
        neighbor_row = (row['id'], neighbor.strip())
        neighbor_data.append(neighbor_row)
#print(neighbor_data)
cursor.executemany(add_neighbor_querry, neighbor_data)
mydb.commit()

In [47]:
#ENCLOSED PRECINCTS

add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

update_enclosed_error_querry = "UPDATE mhaczynska.data_error SET STATE = %s WHERE PRECINCT = %s"

ERROR_ECLOSED_TYPE = 'ENCLOSED'
ERROR_MULTIPOLYGON_TYPE = 'MULTIPOLYGON'
STATE = 'NY'
enclosed_error_data = []
multipolygon_error_data = []
for index, row in precincts.iterrows():    
    if  len(row.geometry) > 1:
        multipolygon_row = (ERROR_MULTIPOLYGON_TYPE, row['id'], row['centerX'], row['centerY'], STATE)
        multipolygon_error_data.append(multipolygon_row)
    for i, polygon in enumerate(row.geometry):
            #if i != 0:
        interior_list = list(polygon.interiors)
        if len(interior_list) > 0:
            bounds = interior_list[0].bounds
            centerX = round((bounds[0] + bounds[2]) / 2, 6)
            centerY = round((bounds[1] + bounds[3]) / 2, 6)
            enclosed_row = (ERROR_ECLOSED_TYPE, row['id'], centerY, centerX, STATE)
            enclosed_error_data.append(enclosed_row)


#print(enclosed_error_data)           
#print(multipolygon_error_data)            
cursor.executemany(add_error_querry, enclosed_error_data)
mydb.commit()
cursor.executemany(add_error_querry, multipolygon_error_data)
mydb.commit()

In [52]:
## OVERLAP 
add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

update_enclosed_error_querry = "UPDATE mhaczynska.data_error SET STATE = %s WHERE PRECINCT = %s and ERROR_TYPE = %s"

file = './NY/ny_intersection_points.geojson'
intersections = gp.read_file(file)
intersections_data = []
ERROR_INTERSECTIONS_TYPE = 'OVERLAP'
STATE = 'NY'
for index, row in intersections.iterrows():
    point = list(row.geometry.coords)
    #intersection_row = (ERROR_INTERSECTIONS_TYPE, row['id'], point[0][1], point[0][0], STATE)
    update_row = (STATE, row['id'], ERROR_INTERSECTIONS_TYPE)
    intersections_data.append(update_row)

cursor.executemany(update_enclosed_error_querry, intersections_data)
mydb.commit()    
#print(intersections_data)

In [ ]:
cursor.close()
mydb.close()

In [34]:
precincts[precincts['id'] == '360011'].head()

,id,canonical,type,centerX,centerY,neighbors,geometry
220,360011,NY Albany 1,ORIGINAL,+42.6470787,-073.7609171,"3600115,3600153,3600127,3600128,360012,3600165...","MULTIPOLYGON (((-73.75804 42.64685, -73.75650 ..."


In [37]:
precincts.shape

(14926, 7)

In [39]:
dup = precincts[precincts.duplicated()]
dup.head()

,id,canonical,type,centerX,centerY,neighbors,geometry
